<a href="https://colab.research.google.com/github/caavila96/Informalidad_GEIH_2022/blob/main/informalidad_GEIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto para la estimación de los determinantes de la informalidad laboral en Colombia.**

Para la generación de este proyecto se usará la base de datos de la Gran Encuesta Integrada de Hogares - GEIH 2022, esta base de datos es publicada por el Departamento Administrativo Nacional de Estadísticas - DANE de forma mensual (anonimizada) en diversos formatos.

Teniendo en cuenta esto se han subido los 12 meses a un respositorio en Github en formato .zip, cada uno de los formatos contiene un total de 9 tablas, a saber

*   Características generales, seguridad social en salud y educación 
*   Datos del hogar y la vivienda.
*   Fuerza de trabajo
*   Migración 
*   No ocupados
*   Ocupados
*   Otras formas de trabajo
*   Otros ingresos e impuestos
*   Tipo de investigación

Dado el objetivo de la investigación solo se tendran en cuenta tres bases, a saber:

*   Características generales, seguridad social en salud y educación 
*   Datos del hogar y la vivienda.
*   Fuerza de trabajo
*   Ocupados





## **1. Librerias a ser utilizadas**

In [50]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import requests #Descargar información Github
import io       #Herramientas principales para trabajar con streams
from zipfile import ZipFile #Apertura de los zip file
from functools import reduce #Usado para pegar las tablas

import warnings
warnings.filterwarnings("ignore", category=UnicodeWarning) #ignorar erroes de UTF-8 y Latin-1

## 2. Impotar información.

Para ello se descarga cada uno de los meses se hace el pegue de las bases, se gudarda en el disco temporal y posteriormente se unen los 12 meses.

In [59]:

#url de Github donde se encuentran los datos
url = "https://github.com/caavila96/Informalidad_GEIH_2022/blob/6e51e78dda7fc86892db064eca494c9f9c64174e/BASE/01.zip?raw=true"

# Descarga de los datos de Github
response = requests.get(url)
data = io.BytesIO(response.content)

# Se define una lista que contiene los nombres de las tablas que serán utilizadas
tables = ['Características generales, seguridad social en salud y educación.dta',
                  'Datos del hogar y la vivienda.dta',
                  'Fuerza de trabajo.dta',
                  'Migración.dta',
                  'Ocupados.dta']

# Se crea una lista donde se cargaran todas las tablas definidas
dfs = []

# Se extraen todos los datos definidos en la lista tables
with ZipFile(data, "r") as zip_ref:
    for file in zip_ref.namelist():
        if any(file.endswith(f) for f in tables):
            with zip_ref.open(file) as f:
                df = pd.read_stata(f, convert_categoricals=False)
                dfs.append(df)

#


In [60]:
#Se crea una lista de data frames con las bases de persona
p_df =[df for j, df in enumerate(dfs) if j != 1]



<ipython-input-60-e8823bc44efd>:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'PERIODO_x', 'PER_x', 'AREA_x', 'HOGAR_x', 'MES_x', 'REGIS_x', 'FEX_C18_x', 'CLASE_x', 'DPTO_x'} in the result is deprecated and will raise a MergeError in a future version.
  p_df = reduce(lambda df1,df2: pd.merge(df1,df2,on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how = "left"),  [df for j, df in enumerate(dfs) if j != 1])


In [ ]:
#Se realiza el pegue de las bases a nivel de persona
p_df = reduce(lambda df1,df2: pd.merge(df1,df2,on=["DIRECTORIO","SECUENCIA_P","ORDEN"], how = "left"),  [df for j, df in enumerate(dfs) if j != 1])

In [67]:
dfs[1]


,DIRECTORIO,SECUENCIA_P,PERIODO,P5000,P5010,P5020,P5030,P5040,P5050,P5070,...,P4030S5,P70,REGIS,MES,DPTO,HOGAR,CLASE,AREA,FEX_C18,PER
0,5000000.0,1.0,20220104.0,3.0,2.0,1.0,1.0,1.0,10.0,1.0,...,1.0,1.0,01,1,08,1.0,1,,1432.463323,2022.0
1,5000001.0,1.0,20220104.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,01,1,08,1.0,1,,1088.796266,2022.0
2,5000002.0,1.0,20220104.0,3.0,1.0,1.0,1.0,1.0,10.0,1.0,...,1.0,1.0,01,1,08,1.0,1,,2066.712422,2022.0
3,5000003.0,1.0,20220104.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,01,1,08,1.0,1,,1834.310454,2022.0
4,5000004.0,1.0,20220104.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,01,1,08,1.0,1,,1920.080408,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26093,5032616.0,1.0,20220101.0,5.0,2.0,1.0,1.0,1.0,10.0,1.0,...,1.0,1.0,01,1,91,1.0,1,,56.527131,2022.0
26094,5032617.0,1.0,20220102.0,3.0,1.0,2.0,1.0,1.0,5.0,2.0,...,1.0,1.0,01,1,91,1.0,1,,103.386936,2022.0
26095,5032618.0,1.0,20220103.0,4.0,2.0,1.0,1.0,1.0,10.0,1.0,...,1.0,1.0,01,1,91,1.0,1,,62.769960,2022.0
26096,5032619.0,1.0,20220103.0,4.0,2.0,1.0,1.0,1.0,10.0,1.0,...,1.0,1.0,01,1,91,1.0,1,,116.569149,2022.0


In [69]:
p_df.columns

Index(['DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'P6016', 'P3271', 'P6030S1',
       'P6030S3', 'P6040', 'P6050', 'P6083',
       ...
       'P3062S5', 'P3062S6', 'P3062S7', 'P3062S8', 'P3062S9', 'P3063S1',
       'P3064S1', 'P3366', 'PER_y', 'REGIS_y'],
      dtype='object', length=349)